In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3316131/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_22_8_0,0.997991,0.765443,0.998807,0.967329,0.995535,0.013432,1.568487,0.019977,0.063976,0.041976,0.200522,0.115898,1.000846,0.120833,170.620163,269.349105,"Hidden Size=[20], regularizer=0.2, learning_ra..."
1,model_22_8_1,0.997991,0.765442,0.998807,0.967329,0.995535,0.013433,1.568487,0.019977,0.063976,0.041977,0.200523,0.115899,1.000846,0.120833,170.620143,269.349085,"Hidden Size=[20], regularizer=0.2, learning_ra..."
2,model_22_8_2,0.997991,0.765442,0.998807,0.967328,0.995535,0.013433,1.568488,0.019978,0.063978,0.041978,0.200523,0.115900,1.000846,0.120834,170.620102,269.349044,"Hidden Size=[20], regularizer=0.2, learning_ra..."
3,model_22_8_3,0.997991,0.765442,0.998807,0.967328,0.995535,0.013433,1.568488,0.019978,0.063978,0.041978,0.200523,0.115901,1.000846,0.120835,170.620091,269.349033,"Hidden Size=[20], regularizer=0.2, learning_ra..."
4,model_22_8_4,0.997991,0.765442,0.998807,0.967328,0.995535,0.013433,1.568488,0.019978,0.063979,0.041978,0.200522,0.115901,1.000846,0.120835,170.620076,269.349018,"Hidden Size=[20], regularizer=0.2, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,model_35_9_8,0.956572,0.741047,0.954606,0.986283,0.985059,0.290403,1.731617,0.079829,0.112582,0.096206,0.969441,0.538891,1.015105,0.561833,188.472968,301.828420,"Hidden Size=[23], regularizer=0.05, learning_r..."
2150,model_7_5_23,0.956534,0.782444,0.920179,0.867828,0.930615,0.290656,1.454799,0.252085,0.827106,0.539596,1.173347,0.539125,1.025443,0.562077,132.471230,211.698158,"Hidden Size=[16], regularizer=0.05, learning_r..."
2152,model_29_9_2,0.956504,0.816955,0.692663,0.910919,0.899696,0.290859,1.224022,0.339805,0.177199,0.258502,1.567680,0.539314,1.016060,0.562273,180.469833,288.949781,"Hidden Size=[22], regularizer=0.2, learning_ra..."
2157,model_34_9_0,0.956468,0.746049,0.992956,0.907539,0.985132,0.291100,1.698174,0.061849,0.087987,0.074918,3.295617,0.539537,1.015142,0.562506,188.468175,301.823626,"Hidden Size=[23], regularizer=0.2, learning_ra..."
